In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/share/software/user/open/py-scipy/1.6.3_py39/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2025-11-29 15:34:48.860796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 15:34:49.084075: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 15:34:53.947804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.

## Scaling all sfe and climate data to be between 0 and 1
https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [2]:
# read in list of arrays for each pixel - these are fully filtered
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_filtered_3mwindow.npz", allow_pickle = True)["sfe_window"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_filtered_3mwindow.npz", allow_pickle = True)["climate_window"]



In [3]:
print(sfe_pre.shape)
print(climate_pre.shape)

(3836373,)
(3836373, 3, 8)


In [4]:
sfe_pre_rs = sfe_pre.reshape(-1,1) # single feature
sfe_pre_rs.shape


(3836373, 1)

In [5]:
# climate data is 3d, needs to be 2d for minmaxscaler
# so reshape and then turn back to original shape after processing
climate_pre_ogshape = climate_pre.shape
print(climate_pre_ogshape)

climate_pre_rs = climate_pre.reshape(-1, climate_pre_ogshape[2]) # (pixel-month pairs * timesteps, 8)
climate_pre_rs.shape

(3836373, 3, 8)


(11509119, 8)

In [6]:
scaler_sfe = MinMaxScaler(feature_range=(0,1))
scaler_climate = MinMaxScaler(feature_range=(0,1))

sfe_pre_scaled = scaler_sfe.fit_transform(sfe_pre_rs)
climate_pre_scaled = scaler_climate.fit_transform(climate_pre_rs)



In [7]:
print(sfe_pre_scaled.min(), sfe_pre_scaled.max())
print(climate_pre_scaled.min(), climate_pre_scaled.max())


0.0 1.0
0.0 1.0000000000000002


In [8]:
sfe_back = scaler_sfe.inverse_transform(sfe_pre_scaled)

sfe_back.shape
print(sfe_back.min(), sfe_back.max())
print(sfe_pre_rs.min(), sfe_pre_rs.max())


-0.4033722536155057 5.223653647385592
-0.40337225361550577 5.223653647385593


In [9]:
# change climate data back to orig shape
climate_pre_scaled = climate_pre_scaled.reshape(climate_pre_ogshape)

In [10]:
climate_pre_scaled = np.array(climate_pre_scaled)
sfe_pre_scaled = np.array(sfe_pre_scaled)

# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_pre_scaled, dtype=object)
climate_obj = np.array(climate_pre_scaled, dtype=object)

In [11]:
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", sfe_pre = sfe_obj)
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", climate_pre =climate_obj)

## training the model

https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/

In [12]:
# reading in sliding window data, was also scaled
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", allow_pickle = True)["sfe_pre"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", allow_pickle = True)["climate_pre"]

In [13]:
# converting from objects
sfe_pre = np.array(sfe_pre, dtype=np.float32)
climate_pre = np.array(climate_pre, dtype=np.float32)


In [14]:
climate_train, climate_test, sfe_train, sfe_test = train_test_split(
    climate_pre, sfe_pre, test_size=0.1, shuffle=False
)

In [15]:
model = Sequential()
model.add(BatchNormalization(input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # input shape is the length of the sequence, each input feature
model.add(LSTM(units=64, return_sequences=False,
          input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # return one output per sequence
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics = ['mae'])
model.summary()

2025-11-29 15:36:32.550252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 15:36:33.145011: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38199 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:8d:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 3, 8)             32        
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 18,785
Trainable params: 18,769
Non-trainable params: 16
_________________________________________________________________


In [16]:
history = model.fit(climate_train, sfe_train, epochs=40, batch_size=64, validation_split=0.1)

predictions = model.predict(climate_test)
np.save("/scratch/users/ashdef/model_out/3month/predictions_scaled.npy", predictions)

predictions_original = scaler_sfe.inverse_transform(predictions)
np.save("/scratch/users/ashdef/model_out/3month/predictions_original.npy", predictions_original)

np.save("/scratch/users/ashdef/model_out/3month/sfe_test_scaled.npy", sfe_test)
    
sfe_test_original = scaler_sfe.inverse_transform(sfe_test)
np.save("/scratch/users/ashdef/model_out/3month/sfe_test_original.npy", sfe_test_original)

model.save("/scratch/users/ashdef/model_out/3month/baseline.keras")

rmse = mean_squared_error(sfe_test_original, predictions_original, squared=False)
mae = mean_absolute_error(sfe_test_original, predictions_original)
r2 = r2_score(sfe_test_original, predictions_original)

print(rmse, mae, r2)


Epoch 1/40


2025-11-29 15:36:53.263692: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


   12/48555 [..............................] - ETA: 3:59 - loss: 0.0942 - mae: 0.2527    

2025-11-29 15:36:54.066448: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


48555/48555 [==============================] - 241s 5ms/step - loss: 0.0024 - mae: 0.0358 - val_loss: 0.0022 - val_mae: 0.0337
Epoch 2/40
48555/48555 [==============================] - 237s 5ms/step - loss: 0.0019 - mae: 0.0314 - val_loss: 0.0018 - val_mae: 0.0312
Epoch 3/40
48555/48555 [==============================] - 241s 5ms/step - loss: 0.0017 - mae: 0.0299 - val_loss: 0.0017 - val_mae: 0.0295
Epoch 4/40
48555/48555 [==============================] - 241s 5ms/step - loss: 0.0016 - mae: 0.0290 - val_loss: 0.0017 - val_mae: 0.0296
Epoch 5/40
48555/48555 [==============================] - 243s 5ms/step - loss: 0.0016 - mae: 0.0285 - val_loss: 0.0016 - val_mae: 0.0286
Epoch 6/40
48555/48555 [==============================] - 242s 5ms/step - loss: 0.0015 - mae: 0.0281 - val_loss: 0.0016 - val_mae: 0.0293
Epoch 7/40
48555/48555 [==============================] - 243s 5ms/step - loss: 0.0015 - mae: 0.0278 - val_loss: 0.0016 - val_mae: 0.0287
Epoch 8/40
48555/48555 [=====================

In [17]:
history.history

{'loss': [0.002430840628221631,
  0.0018686429830268025,
  0.001699434476904571,
  0.0016080003697425127,
  0.001551342778839171,
  0.0015099873999133706,
  0.0014811353757977486,
  0.0014564167940989137,
  0.001437132596038282,
  0.0014209593646228313,
  0.0014051346806809306,
  0.0013973461464047432,
  0.0013857855228707194,
  0.001375108608044684,
  0.0013679343974217772,
  0.0013612739276140928,
  0.0013554600300267339,
  0.0013489151606336236,
  0.0013418616726994514,
  0.0013373139081522822,
  0.0013333765091374516,
  0.0013279755366966128,
  0.0013241305714473128,
  0.0013219018001109362,
  0.0013163223629817367,
  0.0013138757785782218,
  0.0013120744843035936,
  0.0013055994641035795,
  0.0013044928200542927,
  0.0013024040963500738,
  0.0012997230514883995,
  0.001298216637223959,
  0.001295284484513104,
  0.0012919375440105796,
  0.001289590261876583,
  0.0012887761695310473,
  0.0012853560037910938,
  0.0012842364376410842,
  0.0012817757669836283,
  0.0012794923968613148],

In [18]:
np.save("/scratch/users/ashdef/model_out/3month/training_history.npy", history.history)